In [1]:
from keras.layers import *
from keras.models import Model, Sequential, load_model
from keras.losses import logcosh
#from keras import backend as K
#import tensorflow as tf

in_height = 360
in_width =  640

out_height = 720
out_width = 1280

#out_height = in_height
#out_width = in_width

Using TensorFlow backend.


In [2]:
sq1x1 = "squeeze1x1"
exp1x1 = "expand1x1"
exp3x3 = "expand3x3"
relu = "relu_"

from keras.constraints import maxnorm

def fire_module(x, fire_id, channels):
    s_id = 'fire' + str(fire_id) + '/'
    x = Conv2D(channels, (1, 1), padding='same', name=s_id + sq1x1,kernel_constraint=maxnorm(3))(x)
    x = Activation('relu', name=s_id + relu + sq1x1)(x)
    
    left = Conv2D(channels//2, (1, 1), padding='same', name=s_id + exp1x1,kernel_constraint=maxnorm(3))(x)
    left = Activation('relu', name=s_id + relu + exp1x1)(left)

    right = Conv2D(channels//2, (3, 3), padding='same', name=s_id + exp3x3,kernel_constraint=maxnorm(3))(x)
    right = Activation('relu', name=s_id + relu + exp3x3)(right)

    x = concatenate([left, right], axis=3, name=s_id + 'concat')
    return x

def reconstruct_module(x, fire_id, squeeze=16, expand=64):
    s_id = 'fire' + str(fire_id) + '/'
    x = Conv2DTranspose(squeeze, (1, 1), padding='valid', name=s_id + sq1x1,strides=2)(x)
    x = Activation('relu', name=s_id + relu + sq1x1)(x)

    left = Convolution2D(expand, (1, 1), padding='valid', name=s_id + exp1x1)(x)
    left = Activation('relu', name=s_id + relu + exp1x1)(left)

    right = Convolution2D(expand, (3, 3), padding='same', name=s_id + exp3x3)(x)
    right = Activation('relu', name=s_id + relu + exp3x3)(right)

    x = concatenate([left, right], axis=3, name=s_id + 'concat')
    return x

In [ ]:
img_input = Input(shape=(in_height,in_width,3))


x = fire_module(img_input, fire_id=1, channels=4)
x = AveragePooling2D(pool_size=(2,2), name='pool1')(x)
 
x = fire_module(x, fire_id=2, channels=8)
x = AveragePooling2D(pool_size=(2,2), name='pool2')(x)

x = fire_module(x, fire_id=3, channels=16)
x = AveragePooling2D(pool_size=(2,2),padding="same")(x)

out = Flatten()(x)
 
model = Model(img_input, out, name='squeezenet')
model.summary()

model.compile(loss=logcosh,optimizer="adam",metrics=['acc'])

In [3]:
import numpy as np

X_train,y_train = np.load("images.npy"),np.load("encoded_output.npy").astype('int')

In [4]:
from keras.models import load_model

model = load_model("squeeze-net-initial.h5")
model.compile(loss=logcosh,optimizer="adam",metrics=['acc'])

In [ ]:
from IPython.display import clear_output

for i in range(20):    
    hist = model.fit(X_train,y_train,batch_size=64,epochs=50)
    clear_output()
    print(hist)
    model.save("squeeze-net-initial.h5")

Epoch 1/50
888/888 [==============================] - 8s 9ms/step - loss: 26.6761 - acc: 0.2196
Epoch 2/50
888/888 [==============================] - 8s 9ms/step - loss: 26.6978 - acc: 0.2241
Epoch 3/50
888/888 [==============================] - 8s 9ms/step - loss: 26.6751 - acc: 0.2196
Epoch 4/50
888/888 [==============================] - 8s 9ms/step - loss: 26.6407 - acc: 0.2218
Epoch 5/50
888/888 [==============================] - 8s 9ms/step - loss: 26.7924 - acc: 0.2196
Epoch 6/50
888/888 [==============================] - 8s 9ms/step - loss: 26.8701 - acc: 0.2196
Epoch 7/50
888/888 [==============================] - 8s 9ms/step - loss: 26.7037 - acc: 0.2185
Epoch 8/50
888/888 [==============================] - 8s 9ms/step - loss: 26.6596 - acc: 0.2207
Epoch 9/50
888/888 [==============================] - 8s 9ms/step - loss: 26.7135 - acc: 0.2173
Epoch 10/50
888/888 [==============================] - 8s 9ms/step - loss: 26.6857 - acc: 0.2218
Epoch 11/50
888/888 [==================

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

from keras.optimizers import Adam, Adadelta

datagen = ImageDataGenerator(featurewise_center=False, featurewise_std_normalization=False,horizontal_flip=True,vertical_flip=True)
#for X_, Y_ in datagen.flow(X,Y, batch_size=256):
#autoencoder.fit_generator(getData(8),epochs=10,samples_per_epoch=50)
#autoencoder.compile(optimizer='adam', loss='mean_absolute_error',metrics=['acc'])

for X_, Y_ in getData(256):
    #X_,_ = datagen.flow(X_,X_,batch_size=256)
    hist = autoencoder_model.fit(X_, Y_,
                epochs=10,
                batch_size=16,validation_data=(x_valid,y_valid),shuffle=True)
    clear_output()
    val_accuracy = hist.history['val_acc']
    print(val_accuracy)
    autoencoder_model.save('SqueezeNet-%dx%d-%dx%d.h5'%(in_height,in_width,out_height,out_width))
    autoencoder_model.save_weights('SqueezeNet-%dx%d-%dx%d.h5'%(in_height,in_width,out_height,out_width))
    print("Saved model to disk")